In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ======================== CELL 1: DEPENDENCIES ==========================
import subprocess
import sys

print('🔧 Installing dependencies...')
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "pyarrow"], capture_output=True, check=False)
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pyarrow==15.0.2", "keybert", "rank-bm25", "faiss-cpu", "sacremoses"], check=True)
print('✅ Dependencies installed. RESTART kernel and run Cell 2.')


In [1]:
# ===================== CELL 2: IMPORTS & SETUP =====================
import warnings
warnings.filterwarnings("ignore")

import re
import json
import time
import pickle
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Tuple

import numpy as np
import torch
import faiss
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize, sent_tokenize
from rank_bm25 import BM25Okapi

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

# ============================================================================
# CONFIGURATION FOR NUTRITION & LIFESTYLE DOMAIN
# ============================================================================

@dataclass
class DomainConfig:
    name: str = "nutrition"
    dataset_name: str = "medalpaca/medical_meadow_health_advice"
    dataset_split: str = "train"
    chunk_window: int = 3
    chunk_stride: int = 1
    embed_model: str = "sentence-transformers/all-MiniLM-L6-v2"

config = DomainConfig()

print(f"📋 Building index for: {config.name}")
print(f"📦 Dataset: {config.dataset_name}")


2025-11-03 14:06:19.646983: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762178779.862771     104 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762178779.925664     104 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🔧 Using device: cpu
📋 Building index for: nutrition
📦 Dataset: medalpaca/medical_meadow_health_advice


In [2]:
# ===================== CELL 3: DATASET LOADING =====================

def extract_qa_pairs(dataset) -> list:
    """Extract Q&A pairs from MedAlpaca health advice dataset."""
    qa_data = []
    
    for idx, row in enumerate(dataset):
        try:
            question = ""
            answer = ""
            
            # MedAlpaca structure: 'instruction' and 'output'
            if 'instruction' in row and 'output' in row:
                question = str(row['instruction']).strip()
                answer = str(row['output']).strip()
            
            # Alternative structure: 'input' and 'output'
            elif 'input' in row and 'output' in row:
                question = str(row['input']).strip()
                answer = str(row['output']).strip()
            
            # Generic structure
            elif 'question' in row and 'answer' in row:
                question = str(row['question']).strip()
                answer = str(row['answer']).strip()
            
            # Validate and add
            if question and answer and len(answer) > 20:
                qa_data.append({
                    "question": question,
                    "answer": answer,
                    "source_id": idx
                })
                
        except Exception as e:
            if idx < 3:
                print(f"⚠️ Row {idx} skipped: {e}")
            continue
    
    return qa_data

# Load dataset
print(f"📥 Loading {config.dataset_name}...")
dataset = load_dataset(config.dataset_name, split=config.dataset_split)
print(f"✅ Loaded {len(dataset)} rows")

# Print sample row to verify structure
print(f"\n📋 Sample row structure:")
print(f"Keys: {list(dataset[0].keys())}")

# Extract Q&A pairs
print(f"\n🔍 Extracting nutrition & lifestyle Q&A pairs...")
qa_data = extract_qa_pairs(dataset)
print(f"✅ Extracted {len(qa_data)} Q&A pairs")

if len(qa_data) < 100:
    print(f"⚠️ WARNING: Only {len(qa_data)} pairs extracted!")
else:
    print(f"\n📝 Sample nutrition Q&A:")
    if qa_data:
        print(f"Q: {qa_data[0]['question'][:150]}...")
        print(f"A: {qa_data[0]['answer'][:150]}...")


📥 Loading medalpaca/medical_meadow_health_advice...


README.md: 0.00B [00:00, ?B/s]

medical_meadow_health_advice.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/8676 [00:00<?, ? examples/s]

✅ Loaded 8676 rows

📋 Sample row structure:
Keys: ['output', 'instruction', 'input']

🔍 Extracting nutrition & lifestyle Q&A pairs...
✅ Extracted 2199 Q&A pairs

📝 Sample nutrition Q&A:
Q: Question: is this a 2) strong advice, 1) weak advice 0) no advice?...
A: This is a weak advice...


In [5]:
def create_chunks(data: List[Dict], window: int = 3, stride: int = 1, min_chars: int = 10) -> List[Dict]:
    """Create sentence-level chunks from answers."""
    chunks = []
    for item in data:
        text = item.get("answer", "")
        if not text or len(text) < min_chars:  # Lower to 10 chars
            continue

        sentences = sent_tokenize(text)
        if not sentences:
            continue

        if len(sentences) <= window:
            chunks.append({
                "chunk": " ".join(sentences),
                "source_idx": item.get("source_id", -1),
                "chunk_id": len(chunks)
            })
            continue

        for i in range(0, max(1, len(sentences) - window + 1), stride):
            chunks.append({
                "chunk": " ".join(sentences[i:i + window]),
                "source_idx": item.get("source_id", -1),
                "chunk_id": len(chunks)
            })
    return chunks

print(f"🔪 Creating chunks (window={config.chunk_window}, stride={config.chunk_stride}, min_chars=10)...")
chunks = create_chunks(qa_data, window=config.chunk_window, stride=config.chunk_stride, min_chars=10)
print(f"✅ Created {len(chunks)} chunks")

if len(chunks) == 0:
    print("[❌ ERROR] No chunks created! Review the extracted sample answers, min_chars, or dataset format.")
else:
    print("\n📝 Sample chunk:")
    for i, chunk in enumerate(chunks[:3]):
        print(f"{i+1}. {chunk['chunk'][:120]}...")


🔪 Creating chunks (window=3, stride=1, min_chars=10)...
✅ Created 2199 chunks

📝 Sample chunk:
1. This is a weak advice...
2. This is a strong advice...
3. This is a weak advice...


In [6]:
# ===================== CELL 5: BUILD FAISS INDEX =====================

# Load embedder
print(f"📦 Loading embedder: {config.embed_model}")
embedder = SentenceTransformer(config.embed_model, device=device)
print(f"✅ Embedder loaded")

# Extract chunk texts
id2doc = [chunk["chunk"] for chunk in chunks]
print(f"📊 Encoding {len(id2doc)} chunks...")

# Create embeddings
embeddings = embedder.encode(
    id2doc,
    normalize_embeddings=True,
    show_progress_bar=True,
    batch_size=64,
    convert_to_numpy=True
).astype('float32')

print(f"✅ Embeddings shape: {embeddings.shape}")

# Build FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

print(f"✅ FAISS index built: {index.ntotal} vectors, dimension {dim}")


📦 Loading embedder: sentence-transformers/all-MiniLM-L6-v2
✅ Embedder loaded
📊 Encoding 2199 chunks...


Batches:   0%|          | 0/35 [00:00<?, ?it/s]

✅ Embeddings shape: (2199, 384)
✅ FAISS index built: 2199 vectors, dimension 384


In [7]:
# ===================== CELL 6: BUILD BM25 INDEX =====================

# Tokenize for BM25
print(f"🔨 Building BM25 index...")
bm25_corpus = [word_tokenize(doc.lower()) for doc in id2doc]
bm25 = BM25Okapi(bm25_corpus)
print(f"✅ BM25 index built")


🔨 Building BM25 index...
✅ BM25 index built


In [8]:
# ===================== CELL 7: SAVE INDEXES =====================

# File names
index_file = f"{config.name}_faiss.index"
id2doc_file = f"{config.name}_id2doc.pkl"
metadata_file = f"{config.name}_metadata.json"

# Save FAISS index
print(f"💾 Saving FAISS index to {index_file}...")
faiss.write_index(index, index_file)

# Save id2doc mapping
print(f"💾 Saving id2doc to {id2doc_file}...")
with open(id2doc_file, "wb") as f:
    pickle.dump(id2doc, f)

# Save metadata
metadata = {
    "created_at": time.time(),
    "domain": config.name,
    "dataset": config.dataset_name,
    "n_vectors": int(index.ntotal),
    "embedding_dim": dim,
    "chunk_window": config.chunk_window,
    "chunk_stride": config.chunk_stride,
    "embed_model": config.embed_model
}

print(f"💾 Saving metadata to {metadata_file}...")
with open(metadata_file, "w") as f:
    json.dump(metadata, f, indent=2)

print("\n" + "="*80)
print("✅ INDEX BUILDING COMPLETE!")
print("="*80)
print(f"📁 Files created:")
print(f"  - {index_file}")
print(f"  - {id2doc_file}")
print(f"  - {metadata_file}")
print(f"\n📊 Statistics:")
print(f"  - Total vectors: {index.ntotal}")
print(f"  - Embedding dimension: {dim}")
print(f"  - Source Q&A pairs: {len(qa_data)}")
print(f"  - Chunks created: {len(chunks)}")
print("="*80)


💾 Saving FAISS index to nutrition_faiss.index...
💾 Saving id2doc to nutrition_id2doc.pkl...
💾 Saving metadata to nutrition_metadata.json...

✅ INDEX BUILDING COMPLETE!
📁 Files created:
  - nutrition_faiss.index
  - nutrition_id2doc.pkl
  - nutrition_metadata.json

📊 Statistics:
  - Total vectors: 2199
  - Embedding dimension: 384
  - Source Q&A pairs: 2199
  - Chunks created: 2199


In [9]:
# ===================== CELL 8: VERIFY INDEX (OPTIONAL) =====================

def test_retrieval(query: str, top_k: int = 5):
    """Test the built index with a sample query."""
    print(f"\n🔍 Testing query: '{query}'")
    
    # Embed query
    query_emb = embedder.encode([query], normalize_embeddings=True, convert_to_numpy=True).astype('float32')
    
    # Search FAISS
    D, I = index.search(query_emb, top_k)
    
    print(f"\n📋 Top {top_k} results:")
    for i, (idx, score) in enumerate(zip(I[0], D[0]), 1):
        print(f"\n{i}. Score: {score:.4f}")
        print(f"   {id2doc[idx][:200]}...")

# Test queries
test_queries = [
    "What foods are good for heart health?",
    "How much protein should I eat daily?",
    "What are the benefits of a Mediterranean diet?"
]

print("\n" + "="*80)
print("🧪 TESTING NUTRITION INDEX")
print("="*80)

for query in test_queries:
    test_retrieval(query, top_k=3)



🧪 TESTING NUTRITION INDEX

🔍 Testing query: 'What foods are good for heart health?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.1188
   This is a strong advice...

2. Score: 0.1188
   This is a strong advice...

3. Score: 0.1188
   This is a strong advice...

🔍 Testing query: 'How much protein should I eat daily?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.1184
   This is a strong advice...

2. Score: 0.1184
   This is a strong advice...

3. Score: 0.1184
   This is a strong advice...

🔍 Testing query: 'What are the benefits of a Mediterranean diet?'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


📋 Top 3 results:

1. Score: 0.0980
   This is a strong advice...

2. Score: 0.0980
   This is a strong advice...

3. Score: 0.0980
   This is a strong advice...
